In [ ]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import *
from tensorflow.keras import Model
from tensorflow.keras.callbacks import EarlyStopping

import matplotlib.pyplot as plt

print(tf.__version__)
print(tf.executing_eagerly())
print(f"GPU: {tf.test.is_gpu_available()}")

### Make U-Net

In [ ]:
def get_unet(input_shape, n_filters):
    input_layer = Input(shape=input_shape)
    start_neurons = n_filters
    # Downsample
    conv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(input_layer)
    conv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)
    pool1 = Dropout(0.25)(pool1)

    conv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(pool1)
    conv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)
    pool2 = Dropout(0.5)(pool2)

    conv3 = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(pool2)
    conv3 = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(conv3)
    pool3 = MaxPooling2D((2, 2))(conv3)
    pool3 = Dropout(0.5)(pool3)

    conv4 = Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same")(pool3)
    conv4 = Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same")(conv4)
    pool4 = MaxPooling2D((2, 2))(conv4)
    pool4 = Dropout(0.5)(pool4)

    # Middle
    convm = Conv2D(start_neurons * 16, (3, 3), activation="relu", padding="same")(pool4)
    convm = Conv2D(start_neurons * 16, (3, 3), activation="relu", padding="same")(convm)
    
    # Upsample
    deconv4 = Conv2DTranspose(start_neurons * 8, (3, 3), strides=(2, 2), padding="same")(convm)
    uconv4 = concatenate([deconv4, conv4])
    uconv4 = Dropout(0.5)(uconv4)
    uconv4 = Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same")(uconv4)
    uconv4 = Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same")(uconv4)

    deconv3 = Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), padding="same")(uconv4)
    uconv3 = concatenate([deconv3, conv3])
    uconv3 = Dropout(0.5)(uconv3)
    uconv3 = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(uconv3)
    uconv3 = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(uconv3)

    deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(uconv3)
    uconv2 = concatenate([deconv2, conv2])
    uconv2 = Dropout(0.5)(uconv2)
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(uconv2)
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(uconv2)

    deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
    uconv1 = concatenate([deconv1, conv1])
    uconv1 = Dropout(0.5)(uconv1)
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(uconv1)
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(uconv1)
    
    output_layer = Conv2D(1, (1,1), padding="same", activation="sigmoid")(uconv1)
    
    return Model(inputs=[input_layer], outputs=[output_layer])

### Training

In [ ]:
# Define training generator
data_gen_args = dict(rescale = 1./255,
                     featurewise_center=False,
                     featurewise_std_normalization=False,
                     rotation_range=90,
#                      width_shift_range=0.1,
#                      height_shift_range=0.1,
                     zoom_range=0.2,
                     horizontal_flip=True,
                     vertical_flip=True)
image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)
# Provide the same seed and keyword arguments to the flow methods
seed = 1
image_generator = image_datagen.flow_from_directory(
    '/kaggle/input/data/samples',
    class_mode=None,
    seed=seed,
    shuffle=True)
mask_generator = mask_datagen.flow_from_directory(
    '/kaggle/input/data/masks',
    class_mode=None,
    seed=seed,
    shuffle=True)

def train_gen():
    while True:
        img = tf.image.rgb_to_grayscale(next(image_generator))
        mask = tf.image.rgb_to_grayscale(next(mask_generator))
        
        yield (img, mask)

### Validation

In [ ]:
val_seed = 42
val_args = dict(rescale = 1./255)
val_image_datagen = ImageDataGenerator(**val_args)
val_mask_datagen = ImageDataGenerator(**val_args)
val_image_generator = image_datagen.flow_from_directory(
    '/kaggle/input/data/samples',
    class_mode=None,
    seed=val_seed,
    shuffle=True)
val_mask_generator = mask_datagen.flow_from_directory(
    '/kaggle/input/data/masks',
    class_mode=None,
    seed=val_seed,
    shuffle=True)
def val_gen():
    while True:
        img = tf.image.rgb_to_grayscale(next(val_image_generator))
        mask = tf.image.rgb_to_grayscale(next(val_mask_generator))
        
        yield (img, mask)

## Display some training samples

In [ ]:
gen = train_gen()
plt.figure(figsize=(10,20))
for i in range(5):
    img, mask = next(gen)
    img = np.squeeze(img)
    mask = np.squeeze(mask)
    plt.subplot(5,2,2*i+1)
    plt.imshow(img[0])
    plt.subplot(5,2,2*i+2)
    plt.imshow(mask[0])
plt.show()

In [ ]:
# Get Unet
# Number of convolution filters
n_filters = 64
# 256x256 grayscale image
input_shape = (256, 256, 1)
model = get_unet(input_shape, n_filters)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
EPOCHS = 20
STEPS_PER_EPOCH = 100
VAL_STEPS = 14
model_history = model.fit_generator(train_gen(), epochs=EPOCHS,
                          steps_per_epoch=STEPS_PER_EPOCH,
                          validation_data = val_gen(),
                          validation_steps = VAL_STEPS,
                          callbacks=[EarlyStopping(monitor='val_acc', restore_best_weights=True)])

In [ ]:
model.save('aav-segmentation-model.h5')